In [ ]:
#| default_exp climate

Weather: Lets give them something to talk about

In [ ]:
#| export
import numpy as np
import sys
import os
import math
import random

#data
from collections import namedtuple
from dataclasses import dataclass,  field, asdict
from typing import List
from enum import Enum

#Jeremy
from dialoghelper import * 
from fastcore.basics import patch
from fasthtml.common import *
from fasthtml.jupyter import *
import httpx

#custom
import inspect
import copy
import colorsys

from HexMagic.styles import StyleCSS, SVGBuilder, SVGLayer, SVGPatternLoader, preview, app, StyleDemo, LayerAnimation
from HexMagic.primitives import MapCord, MapSize, MapRect, MapPath, Hex, HexGrid, HexWrapper, HexPosition, hexBackground, HexRegion, windy_edge,unique_windy_edge
from HexMagic.terrain import  TerraDemo, Terrain, GeoBounds, ClimatePreset
#from terrainpattern import TerrainPatterns
from HexMagic.weather import TerraDemo
from HexMagic.geology import  SoilSystem, DrainageBasins, Geology

In [ ]:
#| export
from HexMagic.terrainpatterns import TerrainPatterns

In [ ]:
#| export

from collections import deque

## Climates

In [ ]:
#| export
from enum import Enum

class Climate(Enum):
    MARINE = 0        # Ocean/sea
    FRESHWATER = 1    # Lakes/rivers
    TUNDRA = 2        # Cold, low precipitation
    DESERT = 3        # Hot/cold, very low precipitation
    GRASSLAND = 4     # Moderate temp, moderate precipitation
    FOREST = 5        # Moderate temp, high precipitation
    JUNGLE = 6        # Hot, very high precipitation







In [ ]:
#| export
## pass 2
@patch
def compute_climate(self: Terrain,force_recompute=True):
    """Classify climate zones based on temperature and precipitation."""
    n_hexes = len(self.elevations)
    
    if ('temperature' not in self.fields) or force_recompute:
        self.compute_weather(force_recompute=force_recompute)
    if 'precipitation' not in self.fields:
        raise ValueError("Must compute precipitation first")
    
    climate = np.zeros(n_hexes, dtype=int)
    temp = self.fields['temperature']
    precip = self.fields['precipitation']
    
    for i in range(n_hexes):
        elev = self.elevations[i]
        t = temp[i]
        p = precip[i]
        
        # Marine (ocean/sea)
        if elev <= 0:
            climate[i] = Climate.MARINE.value
        
        # Tundra (very cold) - stricter threshold
        elif t < 0:  # Changed from 5 to 0
            climate[i] = Climate.TUNDRA.value
        
        # Desert (very dry)
        elif p < 250:
            climate[i] = Climate.DESERT.value
        
        # Jungle (hot and very wet)
        elif t > 20 and p > 1500:
            climate[i] = Climate.JUNGLE.value
        
        # Forest (good rainfall)
        elif p > 750:  # Changed from 800
            climate[i] = Climate.FOREST.value
        
        # Grassland (moderate conditions)
        elif p >= 250:
            climate[i] = Climate.GRASSLAND.value
        
        # Fallback to desert
        else:
            climate[i] = Climate.DESERT.value
    
    self.fields['climate'] = climate
    return climate


In [ ]:
#| export
@patch
def add_climate_overlay(self: Terrain, layer_name="climate"):
    """Visualize climate zones with appropriate colors."""
    opacity = 0.5
    
    climate_colors = {
        Climate.MARINE.value: "#1e88e5",
        Climate.FRESHWATER.value: "#42a5f5",
        Climate.TUNDRA.value: "#e3f2fd",
        Climate.DESERT.value: "#fdd835",
        Climate.GRASSLAND.value: "#9ccc65",
        Climate.FOREST.value: "#2e7d32",
        Climate.JUNGLE.value: "#1b5e20",
    }
    
    if 'climate' not in self.fields:
        self.compute_climate()
    
    climate_indices = self.fields['climate'].astype(int)
    
    # Create patterns
    patternGen = TerrainPatterns(self)
    colors = [climate_colors[i] for i in range(len(Climate))]
    patterns = patternGen.ballDensity(len(colors), fills=colors, prefix="climate")
    
     # Add opacity to pattern styles
    for pattern in patterns:
        pattern.opacity = opacity  # This may need adjustment based on your StyleCSS implementation
    
    # Generate overlay for all hexes
    overlay = self.makeOverlay(climate_indices, patterns)
    
    self.builder.adjust(layer_name, overlay)


In [ ]:
#| export
@patch
def add_climate_overlay(self: Terrain, layer_name="climate",showLegend=True, scale=0.2,background=None):
    """Visualize climate zones with appropriate colors."""
    patGen = TerrainPatterns(self)
    terrain = self
    sgrid = self.hexGrid
    aRender = sgrid.builder

    terrain.colorMap()
    
    # Find ocean hexes (level 0)
    ocean_hexes = terrain.find_region_at_level(0)
    ocean_region = HexRegion(hexes=ocean_hexes, hexGrid=terrain.hexGrid)
    
    # Create wave pattern with ocean blues
   
    wave = patGen.wavePattern("ocean_waves_pat", 
                              amplitude=4, 
                              wavelength=16, 
                              color="#1565C0",      # stroke: medium blue
                              fill="#E3F2FD")       # fill: light blue

    oceanStyle = StyleCSS("ocean_waves", fill=f"url(#ocean_waves_pat)")

    aRender.add_definition(wave)
    aRender.add_style(oceanStyle)
    

    patterns, styles = patGen.climateStyle(scale=scale,commonFill=background)

       # Add patterns to builder
    for p in patterns:
        self.builder.add_definition(p)

    for s in styles:
        self.builder.add_style(s)
    
    if 'climate' not in self.fields:
        self.compute_climate()
    
    climate_indices = self.fields['climate'].astype(int)
    climateRegions = sgrid.regions_by_value(climate_indices)

    for region in climateRegions:
        idx  =  region.hexes.pop()
        region.hexes.add(idx)
        
        styleI = int(climate_indices[idx])
        style = styles[styleI]
        for i in region:
            sgrid.hexes[i].style = style
            
    #for i in ocean_region:
    #    sgrid.hexes[i].style = oceanStyle
    
    overlay =  sgrid.styleLayerOrdered(
        styles=styles,
        f=unique_windy_edge(iterations=3))
    
    aRender.adjust(layer_name, overlay)
    if showLegend:
        legend = aRender.legendOverlay(styles)
        aRender.adjust("legend", legend)

In [ ]:
#| export
@patch
def downsample_climate(self: Terrain, scale=0.5,sample_radius=1):
    """Downsample terrain with all climate data preserved."""

    new_terrain = self.shrinkWeather(scale=scale,sample_radius=sample_radius)
    new_terrain.geo = self.geo

    
    # Add color styles
    if new_terrain.colorLevels:
        for color in new_terrain.colorLevels:
            new_terrain.hexGrid.builder.add_style(color)
    new_terrain.compute_climate()
    
    return new_terrain


In [ ]:
#| export
@patch
def climateIconMap(self: Geology, scale=20):
    scale = scale / 100

    grid = self.terrain.hexGrid
    builder = grid.builder
    builder.layers = []
    self.terrain.add_climate_overlay(scale=scale, showLegend=False)
    
    builder.adjust("watersheds", self.basins.draw_watersheds())
    
    # Get climate styles for legend
    patGen = TerrainPatterns(self.terrain)
    _, styles = patGen.climateStyle(scale=scale)
    
    # Add legend below the map
    legend_svg = builder.legendBelow(styles, use_hex=True)
    builder.adjust("legend", legend_svg)

    legend_text = f"{self.name} Climate"
    builder.add_centered_text(
        legend_text, 
        y_offset=-builder.height / 2 + 30,
        class_name="watershed_legend"
    )

    return builder.show()


In [ ]:
Geology.simpleWorld().climateIconMap(10)

In [ ]:
Geology.simpleWorld().climateIconMap(scale=5)

In [ ]:
@patch
def caliClimate(self:TerraDemo,debug=True):
    terrain_ca = self.california_map()
    
    terrain_ca.compute_climate()

    smaller = terrain_ca.downsample_climate(sample_radius=2)
    #smaller = terrain_ca
    
    smaller.colorMap()
    #smaller.hexGrid.update()

    if debug:
        smaller.summarize_climate()
    # Visualize the downsampled terrain
    smaller.add_climate_overlay(scale=0.1)
   
    smaller.builder.layers = []
    smaller.hexGrid.update()
    #smaller.hexGrid.builder.adjust("contors", smaller.contorOverlay(commonStroke="black"))
    
    
    return smaller.hexGrid.builder.show()

In [ ]:
TerraDemo().caliClimate(False)

### Pretty pictures

In [ ]:
#| export
@patch
def makeClimateOverlay(self: Terrain, climate_data, precip_data, all_patterns: list[SVGBuilder]) -> str:
    """
    Create overlay for climate+precipitation visualization.
    
    Args:
        climate_data: array of climate type values (0-6)
        precip_data: array of precipitation values (mm/year)
        all_patterns: list of 35 patterns (7 climates * 5 precip levels)
    
    Returns:
        SVG string for the overlay
    """
    testBody = ""
    grid = self.hexGrid
    preset = self.climate
    
    # Track which patterns are actually used
    used_patterns = set()
    
    for i in range(len(climate_data)):
        climate_val = int(climate_data[i])
        precip = precip_data[i]
        precip_level = preset.get_precip_level(precip)
        
        # Calculate flat pattern index
        pattern_idx = climate_val * 5 + precip_level
        
        # Bounds check
        if pattern_idx >= 0 and pattern_idx < len(all_patterns):
            used_patterns.add(pattern_idx)
            
            patName = all_patterns[pattern_idx].attributes['id']
            fill = f"url(#{patName})"
            
            ret = "<polygon points=\""
            hex = grid.hexes[i]
            for point in hex.vertices():
                ret += f"{point.x:.0f},{point.y:.0f} "
            ret += f"\" style=\"fill:{fill}\""
            ret += "/>"
            
            testBody += "\t" + ret + "\n"
    
    # Only add used patterns to builder
    for pattern_idx in sorted(used_patterns):
        grid.builder.add_definition(all_patterns[pattern_idx])
    
    return testBody

In [ ]:
#| export
@patch
def dottedClimate(self: Terrain, 
                  flow_levels: int = 5,
                  min_density: float = 0.25,  # Minimum dot density for dry areas
                  debug: bool = False) -> str:
    """Combine climate colors with precipitation-based dot density.
    
    Args:
        flow_levels: Number of dot density levels (default 5)
        min_density: Minimum density for low-precip areas (0.25 = 25%)
        debug: Print pattern generation info
    
    Returns:
        SVG string for the overlay
    """
    
    if 'climate' not in self.fields or 'precipitation' not in self.fields:
        raise ValueError("Need both climate and precipitation computed")
    
    # Define climate colors (muted palette)
    climate_colors = {
        Climate.MARINE: "#A9B9D4",
        Climate.FRESHWATER: "#7BA3C0",
        Climate.TUNDRA: "#8E7159",
        Climate.DESERT: "#D8A48F",
        Climate.GRASSLAND: "#B9B291",
        Climate.FOREST: "#7A9B76",
        Climate.JUNGLE: "#5D7C5A",
    }
    
    # Generate patterns for each climate type
    patternGen = TerrainPatterns(self)
    all_patterns = []
    
    for climate_type in Climate:
        color = climate_colors.get(climate_type, "#cccccc")
        
        # Create flow_levels density patterns for this climate color
        patterns = patternGen.ballDensity(
            levels=flow_levels + 3,
            fills=[color],
            prefix=f"{climate_type.name.lower()}_ball"
        )[:flow_levels]
        
        if debug:
            print(f"{climate_type.name}: generated {len(patterns)} patterns with color {color}")
        
        all_patterns.extend(patterns)
    
    # Build overlay
    grid = self.hexGrid
    overlay = ""
    used_patterns = set()
    
    climate_data = self.fields['climate']
    precip_data = self.fields['precipitation']
    
    # Find max precipitation for normalization
    max_precip = precip_data.max()
    min_precip = precip_data.min()
    
    if debug:
        print(f"\nPrecipitation range: {min_precip:.0f} - {max_precip:.0f} mm/year")
        print(f"Min density: {min_density * 100}%")
    
    # Generate hex polygons
    for i in range(len(climate_data)):
        climate_val = int(climate_data[i])
        precip = precip_data[i]
        
        # Normalize precipitation to [min_density, 1.0] range
        if max_precip > min_precip:
            normalized_precip = (precip - min_precip) / (max_precip - min_precip)  # [0, 1]
            # Scale to [min_density, 1.0]
            scaled_precip = min_density + normalized_precip * (1.0 - min_density)
            precip_level = int(scaled_precip * (flow_levels - 1))
            precip_level = min(precip_level, flow_levels - 1)
        else:
            precip_level = 0
        
        # Calculate pattern index: climate_val * flow_levels + precip_level
        pattern_idx = climate_val * flow_levels + precip_level
        
        # Bounds check
        if pattern_idx >= 0 and pattern_idx < len(all_patterns):
            used_patterns.add(pattern_idx)
            
            patName = all_patterns[pattern_idx].attributes['id']
            fill = f"url(#{patName})"
            
            # Generate polygon
            hex_obj = grid.hexes[i]
            ret = "<polygon points=\""
            for point in hex_obj.vertices():
                ret += f"{point.x:.0f},{point.y:.0f} "
            ret += f"\" style=\"fill:{fill}\"/>"
            
            overlay += "\t" + ret + "\n"
    
    # Add used patterns to builder
    for pattern_idx in sorted(used_patterns):
        grid.builder.add_definition(all_patterns[pattern_idx])
    
    if debug:
        print(f"\nUsed {len(used_patterns)} patterns out of {len(all_patterns)}")
    
    return overlay


In [ ]:
#| export
@patch
def terrainCream(self: Terrain, layer_name="terrain_base"):
    """Create parchment-style base fills using elevation + climate + coast."""
    
    terrain_fills = {
        'ocean':     '#E3F2FD',  # light blue (or pattern)
        'coast':     '#F8F4E8',  # cooler cream near water
        'lowland':   '#FDF5E6',  # base old lace
        'plains':    '#FAF0D4',  # warmer yellow
        'hills':     '#EFE6D5',  # slightly darker
        'highlands': '#E8DFD0',  # aged/darker
        'mountain':  '#DED4C4',  # even darker for peaks
    }
    
    # Create styles
    styles = {k: StyleCSS(k, fill=v) for k, v in terrain_fills.items()}
    for s in styles.values():
        self.builder.add_style(s)
    
    grid = self.hexGrid
    
    # Ensure we have distance_to_coast
    if 'distance_to_coast' not in self.fields:
        self.compute_distance_to_coast()
    
    for i in range(len(self.elevations)):
        elev = self.elevations[i]
        dist_coast = self.fields['distance_to_coast'][i]
        
        if elev <= 0:
            style = styles['ocean']
        elif dist_coast <= 2 and elev < 200:
            style = styles['coast']
        elif elev < 200:
            style = styles['lowland']
        elif elev < 500:
            style = styles['plains']
        elif elev < 1200:
            style = styles['hills']
        elif elev < 2000:
            style = styles['highlands']
        else:
            style = styles['mountain']
        
        grid.hexes[i].style = style
    
    # Render as base layer
    overlay = grid.styleLayerOrdered(styles=list(styles.values()))
    self.builder.adjust(layer_name, overlay)


In [ ]:
#| export
@patch
def climateDotMap(self:Geology,showHexes=False):

    grid = self.terrain.hexGrid
    builder = grid.builder
    builder.layers = []

    self.terrain.colorMap()
    self.terrain.compute_climate()
    
    self.terrain.terrainCream()

    builder.adjust("climates", self.terrain.dottedClimate())

    terrain_fills = {
        'ocean':     '#E3F2FD',  # light blue (or pattern)
        'coast':     '#F8F4E8',  # cooler cream near water
        'lowland':   '#FDF5E6',  # base old lace
        'plains':    '#FAF0D4',  # warmer yellow
        'hills':     '#EFE6D5',  # slightly darker
        'highlands': '#E8DFD0',  # aged/darker
        'mountain':  '#DED4C4',  # even darker for peaks
    }
    
    # Create styles
    legend = [StyleCSS(k, fill=v) for k, v in terrain_fills.items()]

    #builder.adjust("legend",builder.legendOverlay(legend,width=100))
    builder.adjust("legend", builder.legendBelow(legend, use_hex=True))

    legend_text = f"{self.name} Climate"
    
    self.terrain.hexGrid.builder.add_centered_text(
        legend_text, 
        y_offset=-self.terrain.hexGrid.builder.height/2 + 30,
        class_name="watershed_legend"
    )
    return builder.show()

NameError: name 'patch' is not defined

In [ ]:
Geology.simpleWorld().climateDotMap()

## Factory

In [ ]:
#| export
@patch
def recomputeClimate(self:Terrain):
    self.climate.configure(self)

In [ ]:
#| export

class TerrainFactory:
    """Factory for creating terrains with realistic climate parameters."""
    
    
    @staticmethod
    def create_world(bounds: MapRect, 
                        preset: str = 'temperate',
                        name:str = 'untitled',
                        radius: float = 15,
                        lon_span: float = 10.0,
                        custom_params: dict = None,
                        # Plate params
                        num_plates: int = None, 
                        subdivisions: int = 3,
                        ocean_fraction: float = 0.4,
                        oceanic_sides: list = ['N'],
                        edge_factor: float = 1.5,
                        # Terrain character
                        terrain_age: str = 'middle',  # 'young', 'middle', 'old'
                        formation_type: str = 'ocean_distance',
                        elevation_scale: float = 1.0,
                        # Erosion
                        erosion_age: float = 0.25,
                        num_lakes: int = 4,
                        seed: int = None,
                        debug: bool = False) -> Geology:
        """
        Create a world with tectonic plates and climate.
        
        Args:
            bounds: MapRect for the hex grid
            preset: Climate preset name (see PRESETS)
            radius: Hex radius
            lon_span: Longitude span in degrees
            custom_params: Override specific climate preset parameters
            num_plates: Number of tectonic plates (None for blank ocean)
            subdivisions: Plate subdivision depth for detail
            ocean_fraction: Fraction of plates marked oceanic
            oceanic_sides: List of sides ['N','E','S','W'] that are ocean
            edge_factor: How far from edge to mark as oceanic
            terrain_age: 'young' (sharp), 'middle', 'old' (eroded) - affects initial terrain
            formation_type: 'ocean_distance', 'ridge', 'volcanic', 'rolling'
            elevation_scale: Multiplier for elevations
            erosion_age: Age for erosion model (0-1, higher = more eroded)
            num_lakes: how much do we carve
            seed: Random seed
            debug: Print debug info
        
        Returns:
            Geology with terrain, plates, soil, and erosion model
        """

       
        
       
        
        # Create terrain
        if num_plates is None:
            terrain = Terrain(bounds, radius=radius)
            terrain.elevations = np.zeros(len(terrain.elevations))  # All ocean
            plates = []
        else:
            terrain, plates = Terrain.fromSeeds(
                bounds, 
                radius=radius,
                num_plates=num_plates,
                subdivisions=subdivisions,
                ocean_fraction=ocean_fraction,
                oceanic_sides=oceanic_sides,
                edge_factor=edge_factor,
                age=terrain_age,
                formation_type=formation_type,
                elevation_scale=elevation_scale,
                seed=seed
            )

        presets = TerrainPatterns(terrain).weatherPatterns()
        
        if preset not in presets:
            raise ValueError(f"Unknown preset: {preset}. Available: {list(presets.keys())}")
        
        climate_preset = presets[preset]
        

        terrain.climate = climate_preset

         # Calculate geographic bounds
        lat_min, lat_max = climate_preset.lat_range

    
        lon_min = -lon_span / 2
        lon_max = lon_span / 2


        terrain.geo = GeoBounds(
            lat_min=lat_min,
            lat_max=lat_max,
            lon_min=lon_min,
            lon_max=lon_max
        )

        if num_lakes is not None:
            rivers = terrain.carve_to_ocean( num_lakes=num_lakes)

        world = Geology(terrain,plates,name)

        # Apply custom parameter overrides
        if custom_params:
            for key, value in custom_params.items():
                if hasattr(climate_preset, key):
                    setattr(climate_preset, key, value)
        
        if debug:
            print(f"\n=== TERRAIN FACTORY ===")
            print(f"Preset: {climate_preset.name}")
            print(f"Description: {climate_preset.description}")
            print(f"Latitude: {lat_min}° to {lat_max}°")
            print(f"Longitude: {lon_min}° to {lon_max}°")
            print(f"Grid: {terrain.hexGrid.nRows} x {terrain.hexGrid.nCols} hexes")
            print(f"Terrain age: {terrain_age}, Formation: {formation_type}")
            print(f"Base temperature range: {climate_preset.base_temp_range[0]}°C to {climate_preset.base_temp_range[1]}°C")
            print(f"Wind: {climate_preset.wind_speed} m/s from {climate_preset.wind_dir}°")
        
        return world

    
    
  

In [ ]:

world = TerrainFactory.create_world(
    bounds= MapRect(MapCord(0, 0), MapSize(700, 400)),
    preset='temperate',
    name='Maiden Lane',
    radius=15,
    lon_span=10.0,
    num_plates=8,
    subdivisions=3,
    ocean_fraction=0.3,
    oceanic_sides=['W'],  # Ocean on east and west
    terrain_age='young',  # Sharp, dramatic features
    formation_type='ridge',  # Creates ridge formations
    elevation_scale=1.5,  # Exaggerate the heights
    erosion_age=0.1,  # Minimal erosion for sharp peaks
    num_lakes=0,
    seed=17,
    debug=True
)

# Compute climate
world.update(world.terrain)


# Visualize with rivers carved through the ridge
world.baseMap()

In [ ]:
world.climateDotMap()

In [ ]:
world.climateIconMap(scale=50)

In [ ]:

def island_demo_fixed(debug=False):
    """Create a tropical island with three volcanoes and downsampled rivers."""
    
    # 1. Create blank ocean world with tropical preset
    bounds = MapRect(MapCord(0, 0), MapSize(800, 800))
    world = TerrainFactory.create_world(
        bounds=bounds,
        preset='tropical',
        radius=15,
        lon_span=5.0,
        num_plates=16,
        oceanic_sides=['N'],
        debug = debug
    )
    


    terrain = world.terrain
    if debug:
        print("\n=== COMPUTING CLIMATE ===")
    
    
    
    # 6. Visualize original
    if debug:
        print("\n=== RENDERING ORIGINAL ===")
    terrain.colorMap()
    terrain.hexGrid.update()
    terrain.add_climate_overlay()
    
    
    # 7. Downsample terrain (including flow)
    if debug:
        
        print("\n=== DOWNSAMPLING ===")
    smaller = terrain.downsample_climate(0.5)
    smaller.hexGrid.adjustRadius(20)
    smaller.hexGrid.builder.layers = []
    
    # 9. Visualize downsampled version
    #smaller.colorMap()
    #smaller.hexGrid.update()
    #smaller.add_climate_overlay()

    return smaller, world

isf, isfWorld = island_demo_fixed()
isfWorld.baseMap()


In [ ]:
def showSimple(aMap):
    
    aMap.colorMap()
    sgrid = aMap.hexGrid
    sgrid.builder.layers = []
    sgrid.builder.adjust("regions", sgrid.styleLayerOrdered(
        styles=aMap.colorLevels,
        f=unique_windy_edge(iterations=3)))
    #sampleMap.hexGrid.update()

    return sgrid.builder.show() 
showSimple(isf)

In [ ]:
showSimple(isf.scaled(0.5))